# Experiments for Graph Matching
This notebook contains the experiments to compare the graph matching results via embeddings produced by a GNN to graph matching results via embeddings that are represented by temporal centralities. 

In [2]:
import numpy as np

In [3]:
temporal_betweenness_vectors_sources = np.load('../data/temporal_betweenness_vectors_overtime.npy')
temporal_closeness_vectors_sources = np.load('../data/temporal_closeness_vectors_overtime.npy')
temporal_degree_vectors_sources = np.load('../data/temporal_degree_vectors_overtime.npy')

In [ ]:
# TODO load temporal centralities vectors for targets (by Max)

In [ ]:
# TODO: load encoding vectors from GNN (by Til) for sources and targets

In [ ]:
num_sources = len(temporal_betweenness_vectors_sources)
assert len(temporal_closeness_vectors_sources) == len(temporal_degree_vectors_sources) and len(temporal_closeness_vectors_sources) == num_sources
# TODO: do the same for targets and other sanity checks

## Comparison of GNN to temporal betweenness

In [ ]:
# TODO: do the graph matching ranking (as Til implemented), compare rankings of GNN and TCMs (comparison of top-k via DCG where the TCM is the truth?)
matching_scores_GNN_betweenness = []

for source_idx in range(num_sources):
    source_temporal_betweenness = temporal_betweenness_vectors_sources[source_idx]
    # TODO: do the graph matching for betweenness

    source_GNN = GNN_vectors_sources[source_idx]
    # TODO: do the graph matching for GNN

    # TODO: compare both matchings and compute a score
    matching_scores_GNN_betweenness.append(score)
# TODO: somehow aggregate the scores and generate a total score (average? histograms?)



In [ ]:
# TODO: do the same for closeness and degree

In [ ]:
# TODO: compare scores for closeness, betweenness and degree and find out something meaningful (regarding the use cases)